# **Experiment B1**: proof of concept local refinement
Riemannian gradient descent with local regular integration

Vary noise level and number of images

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/PhD/Projects/2 - Cryo-EM/src/Cryo-EM

[Errno 2] No such file or directory: '/content/drive/MyDrive/PhD/Projects/2 - Cryo-EM/src/Cryo-EM'
/Users/wdiepeveen/Documents/PhD/Projects/2 - Cryo-EM/src/src/Cryo-EM/projects/rkhs_lifting/experiments/experimentB1


In [2]:
# install libraries in Colab env
!pip install mrcfile
!pip install aspire
!pip install quaternionic
!pip install pykeops[colab] > install.log

You should consider upgrading via the '/Users/wdiepeveen/PycharmProjects/Cryo-EM/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/wdiepeveen/PycharmProjects/Cryo-EM/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wdiepeveen/PycharmProjects/Cryo-EM/bin/python -m pip install --upgrade pip' command.
zsh:1: no matches found: pykeops[colab]


In [1]:
%cd ../../../..

/Users/wdiepeveen/Documents/PhD/Projects/2 - Cryo-EM/src/src/Cryo-EM


In [2]:
import numpy as np
import os
import logging

from projects.rkhs_lifting.experiments.experimentB1.E1_run import run_experiment
from projects.rkhs_lifting.experiments.experimentB1.E2_post_processing import post_processing

from tools.exp_tools import Exp

logger = logging.getLogger(__name__)

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.


In [3]:
# Set data path
data_dir = os.path.join("results","expA2_21-10-27_10-39-37_L65_N512_r2_k90_lamV10_lamD-10")
data_filename = "solver_data_2SNR_512N"
# data_path = os.path.join(data_dir, data_filename)

# Set results folder if skip_preprocessing
# results_folder = "results/lifting_21-04-21_11-09-45"  # "results/lifting_21-04-26_12-22-06"

In [4]:
# TODO load data from experiment A: initial volume and images (gt rots)
# TODO select a subset of these images to test the algorithm over

# Experiment parameters
img_size = 65  # was 65 before and was 129 in rotation estimation paper
snr = 1 / 2  # [1/2, 1/4, 1/8, 1/16, 1/32, 1/64, 1/128, 1/256]
num_imgs = 512  # 512 capped
k = 90
l = 9
kernel_radius = np.pi/k  # radius of kernel
sep_dist = 2 * kernel_radius / (l-1)
volume_reg_param = 1e10
rots_batch_size = 8192

gd_step_size = 1.  # TODO test in run_
stop_rots_gd = 5

# Directly start experiment
exp = Exp()

exp.begin(prefix="expB1" , postfix="L{}_N{}_l{}_k{}_lamV{}".format(img_size,num_imgs,l,k,int(np.log(volume_reg_param)/np.log(10))))
exp.dbglevel(4)

2021-10-27 21:21:27,215 INFO Results folder is results/expB1_21-10-27_21-21-27_L65_N512_l9_k90_lamV10


In [ ]:
# Experiment

logger.info("Start Experiment")
logger.info("Running Experiment for SNR = {} and {} images".format(snr, num_imgs))

run_experiment(exp=exp,
               num_imgs=num_imgs,
               snr=snr,
               img_size=img_size,
               kernel_radius=kernel_radius,
               l=l,
               sep_dist=sep_dist,
               gd_step_size=gd_step_size,
               volume_reg_param=volume_reg_param,
               data_dir=data_dir,
               data_filename=data_filename,
               )

2021-10-27 21:21:27,227 INFO Start Experiment
2021-10-27 21:21:27,228 INFO Running Experiment for SNR = 0.5 and 512 images
2021-10-27 21:21:27,230 INFO This experiment illustrates orientation refinement using a lifting approach
2021-10-27 21:21:28,906 INFO Load 3D map and downsample 3D map to desired grids of 65 x 65 x 65.
Start solver
Start Riemannian Gradient Descent Solver
2021-10-27 21:21:29,035 INFO Trying NFFT backend finufft
2021-10-27 21:21:29,051 INFO NFFT backend finufft usable.
2021-10-27 21:21:29,052 INFO Trying NFFT backend cufinufft
2021-10-27 21:21:29,055 INFO NFFT backend cufinufft not usable:
	No module named 'pycuda'
2021-10-27 21:21:29,055 INFO Trying NFFT backend pynfft
2021-10-27 21:21:29,057 INFO NFFT backend pynfft not usable:
	No module named 'pynfft'
2021-10-27 21:21:29,058 INFO Selected NFFT backend = finufft.
2021-10-27 21:21:29,800 INFO ==================== Image 1 ====================
2021-10-27 21:21:29,800 INFO 1 | k = 0 | relative cost = 1
2021-10-27 21:

2021-10-27 21:22:59,715 INFO 11 | k = 3 | relative cost = 0.9940319061279297 | ||gradient|| = 0.00818352691083144
2021-10-27 21:23:01,349 INFO 11 | k = 4 | relative cost = 0.9937641024589539 | ||gradient|| = 0.005598256029423832
2021-10-27 21:23:03,056 INFO 11 | k = 5 | relative cost = 0.9936281442642212 | ||gradient|| = 0.003960137307526524
2021-10-27 21:23:03,856 INFO ==================== Image 12 ====================
2021-10-27 21:23:03,857 INFO 12 | k = 0 | relative cost = 1
2021-10-27 21:23:05,498 INFO 12 | k = 1 | relative cost = 0.9956056475639343 | ||gradient|| = 0.020609209773652912
2021-10-27 21:23:07,498 INFO 12 | k = 2 | relative cost = 0.9933727383613586 | ||gradient|| = 0.014592894397044184
2021-10-27 21:23:09,324 INFO 12 | k = 3 | relative cost = 0.9921895861625671 | ||gradient|| = 0.010627050394547775
2021-10-27 21:23:10,833 INFO 12 | k = 4 | relative cost = 0.9915540218353271 | ||gradient|| = 0.007745546455419206
2021-10-27 21:23:12,365 INFO 12 | k = 5 | relative cost 

2021-10-27 21:24:36,243 INFO ==================== Image 23 ====================
2021-10-27 21:24:36,244 INFO 23 | k = 0 | relative cost = 1
2021-10-27 21:24:37,701 INFO 23 | k = 1 | relative cost = 0.9986982345581055 | ||gradient|| = 0.014136777835019765
2021-10-27 21:24:39,151 INFO 23 | k = 2 | relative cost = 0.9984626770019531 | ||gradient|| = 0.005772146103084874
2021-10-27 21:24:40,611 INFO 23 | k = 3 | relative cost = 0.9983967542648315 | ||gradient|| = 0.0029517625741680934
2021-10-27 21:24:42,073 INFO 23 | k = 4 | relative cost = 0.9983725547790527 | ||gradient|| = 0.0017432964774624718
2021-10-27 21:24:43,524 INFO 23 | k = 5 | relative cost = 0.9983623623847961 | ||gradient|| = 0.0011178215464962022
2021-10-27 21:24:44,259 INFO ==================== Image 24 ====================
2021-10-27 21:24:44,260 INFO 24 | k = 0 | relative cost = 1
2021-10-27 21:24:45,719 INFO 24 | k = 1 | relative cost = 0.9974316358566284 | ||gradient|| = 0.016417047976320907
2021-10-27 21:24:47,177 INF

2021-10-27 21:26:09,036 INFO 34 | k = 3 | relative cost = 0.9983009099960327 | ||gradient|| = 0.0035584752380533147
2021-10-27 21:26:10,505 INFO 34 | k = 4 | relative cost = 0.9982140064239502 | ||gradient|| = 0.002602469501499526
2021-10-27 21:26:11,968 INFO 34 | k = 5 | relative cost = 0.9981557130813599 | ||gradient|| = 0.0021019643484266022
2021-10-27 21:26:12,704 INFO ==================== Image 35 ====================
2021-10-27 21:26:12,705 INFO 35 | k = 0 | relative cost = 1
2021-10-27 21:26:14,165 INFO 35 | k = 1 | relative cost = 0.997919499874115 | ||gradient|| = 0.014901606982148125
2021-10-27 21:26:15,618 INFO 35 | k = 2 | relative cost = 0.9973755478858948 | ||gradient|| = 0.007388641663717079
2021-10-27 21:26:17,074 INFO 35 | k = 3 | relative cost = 0.9971423149108887 | ||gradient|| = 0.004575223708240531
2021-10-27 21:26:18,538 INFO 35 | k = 4 | relative cost = 0.9969977736473083 | ||gradient|| = 0.0035252812676146777
2021-10-27 21:26:20,003 INFO 35 | k = 5 | relative co

2021-10-27 21:27:41,145 INFO ==================== Image 46 ====================
2021-10-27 21:27:41,145 INFO 46 | k = 0 | relative cost = 1
2021-10-27 21:27:42,591 INFO 46 | k = 1 | relative cost = 0.9985238313674927 | ||gradient|| = 0.009663098693512208
2021-10-27 21:27:44,049 INFO 46 | k = 2 | relative cost = 0.9978671073913574 | ||gradient|| = 0.006189823786844513
2021-10-27 21:27:45,505 INFO 46 | k = 3 | relative cost = 0.997490644454956 | ||gradient|| = 0.004643770978272505
2021-10-27 21:27:46,961 INFO 46 | k = 4 | relative cost = 0.9972630739212036 | ||gradient|| = 0.0036066831776291304
2021-10-27 21:27:48,408 INFO 46 | k = 5 | relative cost = 0.997124969959259 | ||gradient|| = 0.0028113826354744034
2021-10-27 21:27:49,137 INFO ==================== Image 47 ====================
2021-10-27 21:27:49,138 INFO 47 | k = 0 | relative cost = 1
2021-10-27 21:27:50,603 INFO 47 | k = 1 | relative cost = 0.9960817098617554 | ||gradient|| = 0.01761319061264596
2021-10-27 21:27:52,065 INFO 47

...

2021-10-27 21:36:59,701 INFO ==================== Image 115 ====================
2021-10-27 21:36:59,702 INFO 115 | k = 0 | relative cost = 1
2021-10-27 21:37:01,164 INFO 115 | k = 1 | relative cost = 0.9973669648170471 | ||gradient|| = 0.021801188886667364
2021-10-27 21:37:02,617 INFO 115 | k = 2 | relative cost = 0.9968251585960388 | ||gradient|| = 0.008472048621982965
2021-10-27 21:37:04,087 INFO 115 | k = 3 | relative cost = 0.99657142162323 | ||gradient|| = 0.00578476327737364
2021-10-27 21:37:05,548 INFO 115 | k = 4 | relative cost = 0.996450662612915 | ||gradient|| = 0.003980369638236908
2021-10-27 21:37:07,014 INFO 115 | k = 5 | relative cost = 0.9963921308517456 | ||gradient|| = 0.0027652255933374717
2021-10-27 21:37:07,739 INFO ==================== Image 116 ====================
2021-10-27 21:37:07,740 INFO 116 | k = 0 | relative cost = 1
2021-10-27 21:37:09,202 INFO 116 | k = 1 | relative cost = 0.9982720613479614 | ||gradient|| = 0.01539596005813261
2021-10-27 21:37:10,653 

2021-10-27 21:38:31,698 INFO 126 | k = 2 | relative cost = 0.9978816509246826 | ||gradient|| = 0.009727120135953636
2021-10-27 21:38:33,286 INFO 126 | k = 3 | relative cost = 0.9973693490028381 | ||gradient|| = 0.0076616365531394694
2021-10-27 21:38:34,779 INFO 126 | k = 4 | relative cost = 0.9970278739929199 | ||gradient|| = 0.0062386094342167546
2021-10-27 21:38:36,268 INFO 126 | k = 5 | relative cost = 0.9968003630638123 | ||gradient|| = 0.005111455447221007
2021-10-27 21:38:37,016 INFO ==================== Image 127 ====================
2021-10-27 21:38:37,017 INFO 127 | k = 0 | relative cost = 1
2021-10-27 21:38:38,504 INFO 127 | k = 1 | relative cost = 0.9994893074035645 | ||gradient|| = 0.0073790753451553025
2021-10-27 21:38:40,003 INFO 127 | k = 2 | relative cost = 0.9992196559906006 | ||gradient|| = 0.005308741862446537
2021-10-27 21:38:41,498 INFO 127 | k = 3 | relative cost = 0.9990694522857666 | ||gradient|| = 0.00393175702187272
2021-10-27 21:38:43,008 INFO 127 | k = 4 | r

2021-10-27 21:40:04,751 INFO 137 | k = 5 | relative cost = 0.9950475096702576 | ||gradient|| = 0.004850243858856042
2021-10-27 21:40:05,486 INFO ==================== Image 138 ====================
2021-10-27 21:40:05,487 INFO 138 | k = 0 | relative cost = 1
2021-10-27 21:40:06,940 INFO 138 | k = 1 | relative cost = 0.9972503781318665 | ||gradient|| = 0.01666699764500281
2021-10-27 21:40:08,413 INFO 138 | k = 2 | relative cost = 0.9960711598396301 | ||gradient|| = 0.010734921983910378
2021-10-27 21:40:09,869 INFO 138 | k = 3 | relative cost = 0.9954735040664673 | ||gradient|| = 0.007499426115924824
2021-10-27 21:40:11,327 INFO 138 | k = 4 | relative cost = 0.9951265454292297 | ||gradient|| = 0.0056535138315721625
2021-10-27 21:40:12,791 INFO 138 | k = 5 | relative cost = 0.9949096441268921 | ||gradient|| = 0.0044394619877688585
2021-10-27 21:40:13,547 INFO ==================== Image 139 ====================
2021-10-27 21:40:13,548 INFO 139 | k = 0 | relative cost = 1
2021-10-27 21:40:15

2021-10-27 21:41:38,609 INFO 149 | k = 1 | relative cost = 0.998080849647522 | ||gradient|| = 0.015746524829616988
2021-10-27 21:41:40,097 INFO 149 | k = 2 | relative cost = 0.9971110224723816 | ||gradient|| = 0.010832948959173819
2021-10-27 21:41:41,601 INFO 149 | k = 3 | relative cost = 0.9964966773986816 | ||gradient|| = 0.008490463296389342
2021-10-27 21:41:43,142 INFO 149 | k = 4 | relative cost = 0.9960696697235107 | ||gradient|| = 0.007036762167141061
2021-10-27 21:41:44,683 INFO 149 | k = 5 | relative cost = 0.9957621097564697 | ||gradient|| = 0.005966300808441343
2021-10-27 21:41:45,450 INFO ==================== Image 150 ====================
2021-10-27 21:41:45,451 INFO 150 | k = 0 | relative cost = 1
2021-10-27 21:41:46,923 INFO 150 | k = 1 | relative cost = 0.9974910616874695 | ||gradient|| = 0.02250053550753925
2021-10-27 21:41:48,478 INFO 150 | k = 2 | relative cost = 0.9964697360992432 | ||gradient|| = 0.012453986082024105
2021-10-27 21:41:49,988 INFO 150 | k = 3 | relat

2021-10-27 21:43:12,769 INFO 160 | k = 4 | relative cost = 0.9942588210105896 | ||gradient|| = 0.007542987063044546
2021-10-27 21:43:14,233 INFO 160 | k = 5 | relative cost = 0.993698239326477 | ||gradient|| = 0.006399732302020297
2021-10-27 21:43:14,960 INFO ==================== Image 161 ====================
2021-10-27 21:43:14,961 INFO 161 | k = 0 | relative cost = 1
2021-10-27 21:43:16,423 INFO 161 | k = 1 | relative cost = 0.9961553812026978 | ||gradient|| = 0.02512057689779187
2021-10-27 21:43:17,879 INFO 161 | k = 2 | relative cost = 0.9948561191558838 | ||gradient|| = 0.014284373959440995
2021-10-27 21:43:19,336 INFO 161 | k = 3 | relative cost = 0.9943417310714722 | ||gradient|| = 0.008941910991349864
2021-10-27 21:43:20,790 INFO 161 | k = 4 | relative cost = 0.9941235184669495 | ||gradient|| = 0.005768645974426661
2021-10-27 21:43:22,256 INFO 161 | k = 5 | relative cost = 0.9940180778503418 | ||gradient|| = 0.00394924400627377
2021-10-27 21:43:22,984 INFO ====================

In [ ]:
logger.info("Start Postprocessing")
# Postprocessing
post_processing(exp=exp,
               num_imgs=num_imgs,
               snr=snr,
              #  results_folder=results_folder #?
               )
